# Generic playspace

## YAML description

In [2]:
import intake

cat = intake.open_df_catalog(
    "/g/data/tm70/intake/dfcatalog.csv",
    yaml_column="yaml",
    name_column="subcatalog",
    columns_with_iterables=["variable"],
)

In [4]:
print(cat.yaml())

sources:
  null:
    args:
      columns_with_iterables:
      - variable
      name_column: subcatalog
      path: /g/data/tm70/intake/dfcatalog.csv
      yaml_column: yaml
    description: ''
    driver: intake_dataframe_catalog.core.DFFileCatalog
    metadata: {}



## Parsing `file_id`

In [117]:
import re

def strip_pattern_rh(patterns, string):
    """
    Sequentially strip a list of regex patterns from a string, starting from the right
    hand side. Then replace any "-" and "." with "_" and then remove double or dangling "_".

    Parameters:
    -----------
    patterns: list of str
        The list of regex patterns to strip
    string: str
        A filename with the suffixe (e.g. .nc) removed
    """

    # Strip first matched pattern
    stripped = string
    for pattern in patterns:
        match = re.match(r".*("+pattern+r")([^0-9]|$).*$", stripped)
        if match:
            stripped = stripped[:match.start(1)] + stripped[match.end(1):]
            break
            
    # Enforce Python characters
    stripped = re.sub(r"[-.]", "_", stripped)
    
    # Remove any double or dangling _
    return re.sub(r"__", "_", stripped).rstrip("_")

In [118]:
cm2_files = [
    "bz687a.pm107912_mon.nc",
    "bz687a.p7107912_mon.nc",
    "iceh_m.2014-06.nc",
    "iceh.1917-05-daily.nc",
    "ocean_bgc_ann.nc-01551231",
    "ocean_bgc_daily.nc-02971231",
    "ocean_bgc_mth.nc-03011231",
    "ocean_bgc.nc-02011231",
    "ocean_daily.nc-02531231",
    "ocean_month.nc-01991231",
    "ocean_scalar.nc-01851231",
]

esm_files = [
    "PI-GWL-B2035.pe-109904_dai.nc",
    "PI-GWL-B2035.pa-109904_mon.nc",
    "PI-1pct-02.pe-011802_dai.nc_dai.nc",
    "iceh.1917-05.nc",
    "iceh.1917-05-daily.nc",
    "ocean_bgc_ann.nc-01551231",
    "ocean_bgc_daily.nc-02971231",
    "ocean_bgc_mth.nc-03011231",
    "ocean_bgc.nc-02011231",
    "ocean_daily.nc-02531231",
    "ocean_month.nc-01991231",
    "ocean_scalar.nc-01851231",
]

om2_files = [
    "iceh.057-daily.nc",
    "ocean.nc",
    "ocean_month.nc",
    "oceanbgc-3d-caco3-1-yearly-mean-y_2015.nc",
    "oceanbgc-2d-wdet100-1-daily-mean-y_2015.nc",
    "ocean-3d-v-1-monthly-pow02-ym_1958_04.nc",
    "ocean-2d-sfc_salt_flux_restore-1-monthly-mean-ym_1958_04.nc",
]

In [121]:
import re
from pathlib import Path

for file in cm2_files:
    
    filename = Path(file).stem
    
    file_id = strip_pattern_rh(["\d{4}[-_]\d{2}", "\d{8}", "\d{6}"], filename)
    #file_id = strip_pattern_rh([r"\d{4}[-_]\d{2}", r"\d{4}", r"\d{3}"], filename)

    print(file_id)

bz687a_pm_mon
bz687a_p7_mon
iceh_m
iceh_daily
ocean_bgc_ann
ocean_bgc_daily
ocean_bgc_mth
ocean_bgc
ocean_daily
ocean_month
ocean_scalar
